<h1> Channel Exploration & Feature Reduction Plan </h1> 

In [ ]:
import pandas as pd
import numpy as np
import re
from pathlib import Path
from collections import Counter
import duckdb

In [4]:
# Ruta fija a DuckDB (la que nos diste)
DB_PATH = Path(r"C:\Users\serrios\01_prueba_analitico_4\op_cobro\database\analytics.duckdb")

# Conectar (solo lectura para exploración)
con = duckdb.connect(str(DB_PATH), read_only=True)

# 1) Obtener SOLO el esquema (nombres de columnas) -> ideal para análisis del patrón de nombres
df_cols = con.execute("DESCRIBE raw.canales").df()
fields = df_cols["column_name"].astype(str).tolist()

print("✅ Total columnas en raw.canales:", len(fields))
print("Ejemplo columnas:", fields[:20])

# 2) (Opcional) traer una muestra de filas para ver comportamiento de un cliente
# OJO: con +4000 columnas, esto puede ser pesado; por eso dejo LIMIT pequeño.
df_canales_sample = con.execute("""
    SELECT *
    FROM raw.canales
    LIMIT 200
""").df()

df_canales_sample.head()


✅ Total columnas en raw.canales: 4258
Ejemplo columnas: ['num_doc', 'obl17', 'f_analisis', 'trx_mnt_total', 'trx_mnt_total_smmlv', 'trx_cnt_total', 'trx_cnt_app_per_pag', 'trx_cnt_app_per_pag_ob', 'trx_cnt_app_per_tfy', 'trx_cnt_app_per_trf', 'trx_cnt_app_per_ava', 'trx_cnt_app_per_ut_c_cr', 'trx_cnt_app_pyme_trf', 'trx_cnt_bill_mvl_cmpr', 'trx_cnt_btn_bco_trf', 'trx_cnt_cajero_pag', 'trx_cnt_cajero_ava', 'trx_cnt_cajero_ret', 'trx_cnt_cajero_trf', 'trx_cnt_cr_bcrio_ret']


,num_doc,obl17,f_analisis,trx_mnt_total,trx_mnt_total_smmlv,trx_cnt_total,trx_cnt_app_per_pag,trx_cnt_app_per_pag_ob,trx_cnt_app_per_tfy,trx_cnt_app_per_trf,...,sum_trx_mnt_gr_rcg_smmlv_ult12,sum_trx_mnt_gr_trf_smmlv_ult12,sum_trx_mnt_gr_pag_ob_smmlv_ult12,sum_trx_mnt_gr_tfy_smmlv_ult12,sum_trx_mnt_gr_ava_smmlv_ult12,sum_trx_mnt_gr_ut_c_cr_smmlv_ult12,sum_trx_mnt_gr_cmpr_smmlv_ult12,sum_trx_mnt_gr_dep_smmlv_ult12,sum_trx_mnt_gr_ret_smmlv_ult12,sum_trx_mnt_gr_dvs_smmlv_ult12
0,922349123250112,5.098631e+16,2025-09-01,909800.0,0.639129,7.0,0.0,0.0,0.0,3.0,...,0.0,11.012188,1.073549,0.0,0.0,0.0,1.418546,0.0,9.808957,0.0
1,899156804630643,7.053737e+16,2024-08-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,215595954901092,8.059354e+16,2024-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,603989110655846,5.839071e+16,2025-11-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,280191780739508,1.735048e+16,2025-07-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# 1) Columnas del dataframe
columns = list(df_canales_sample.columns)

# 2) Llaves (ajusta si aplica)
KEYS = {"num_doc", "obl17", "f_analisis"}

feature_cols = [c for c in columns if c not in KEYS]

print("Total columnas en sample:", len(columns))
print("Total features (sin llaves):", len(feature_cols))
print("Ejemplo:", feature_cols[:25])

Total columnas en sample: 4258
Total features (sin llaves): 4255
Ejemplo: ['trx_mnt_total', 'trx_mnt_total_smmlv', 'trx_cnt_total', 'trx_cnt_app_per_pag', 'trx_cnt_app_per_pag_ob', 'trx_cnt_app_per_tfy', 'trx_cnt_app_per_trf', 'trx_cnt_app_per_ava', 'trx_cnt_app_per_ut_c_cr', 'trx_cnt_app_pyme_trf', 'trx_cnt_bill_mvl_cmpr', 'trx_cnt_btn_bco_trf', 'trx_cnt_cajero_pag', 'trx_cnt_cajero_ava', 'trx_cnt_cajero_ret', 'trx_cnt_cajero_trf', 'trx_cnt_cr_bcrio_ret', 'trx_cnt_cr_bcrio_trf', 'trx_cnt_pos_cmpr', 'trx_cnt_pos_pag', 'trx_cnt_pos_ret', 'trx_cnt_pse_emp_cmpr', 'trx_cnt_pse_per_cmpr', 'trx_cnt_rec_e_v_cmpr', 'trx_cnt_rec_e_v_pag']


In [12]:
# Prefijo = antes del primer "_"
prefixes = [c.split("_")[0] for c in feature_cols if "_" in c]
print("Top prefijos:", Counter(prefixes).most_common(15))

# Ventanas = token final si está en ult3/ult6/ult9/ult12
windows = [c.split("_")[-1] for c in feature_cols if c.split("_")[-1] in {"ult3","ult6","ult9","ult12"}]
print("Ventanas detectadas:", Counter(windows))

# smmlv
smmlv_cols = [c for c in feature_cols if "smmlv" in c.split("_")]
print("Columnas con smmlv:", len(smmlv_cols))
print("Ejemplo smmlv:", smmlv_cols[:10])

Top prefijos: [('avg', 816), ('max', 816), ('stddev', 816), ('sum', 816), ('min', 789), ('trx', 202)]
Ventanas detectadas: Counter({'ult3': 1017, 'ult6': 1017, 'ult9': 1011, 'ult12': 1008})
Columnas con smmlv: 1418
Ejemplo smmlv: ['trx_mnt_total_smmlv', 'trx_mnt_app_per_pag_smmlv', 'trx_mnt_app_per_pag_ob_smmlv', 'trx_mnt_app_per_tfy_smmlv', 'trx_mnt_app_per_trf_smmlv', 'trx_mnt_app_per_ava_smmlv', 'trx_mnt_app_per_ut_c_cr_smmlv', 'trx_mnt_app_pyme_trf_smmlv', 'trx_mnt_bill_mvl_cmpr_smmlv', 'trx_mnt_btn_bco_trf_smmlv']


In [13]:
CHANNEL_DICT = {
    "gr": "gr",
    "cajero": "cajero",
    "app_per": "app personas",
    "pos": "pos",
    "suc_fis": "sucursal fisica",
    "svp": "sucursal virtual personas",
    "cr_bcrio": "cr_bcrio",
    "rec_e_v": "rec_e_v",
    "sve": "sucursal virtual empresa",
    "app_per_pag": "app_per_pag",
    "app_per_tfy": "app_per_tfy",
    "app_per_ut_c": "app_per_ut_c",
    "app_pyme": "app pymes",
    "bill_mvl": "billetera movil",
    "btn_bco": "boton banco",
    "pse_emp": "pse empresas",
    "pse_per": "pse personas",
    "suc_fis_dep": "suc_fis_dep",
    "suc_tel_pag": "sucursal telefonica",
    "suc_tel_ut_c": "suc_tel_ut_c",
    "sv_pyme": "sucursal virtual pymes",
    "sve_dvs": "sve_dvs",
    "sve_pag": "sve_pag",
    "sve_ut_c": "sve_ut_c",
    "svp_dvs": "svp_dvs",
}

KNOWN_CHANNELS = sorted(CHANNEL_DICT.keys(), key=len, reverse=True)

STATS = {"trx", "avg", "max", "min", "stddev", "sum"}
WINDOWS = {"ult3","ult6","ult9","ult12"}

In [14]:
def parse_col(name: str):
    toks = name.split("_")

    # window
    window = toks[-1] if toks and toks[-1] in WINDOWS else "none"
    if window != "none":
        toks = toks[:-1]

    # smmlv
    is_smmlv = "smmlv" in toks
    toks = [t for t in toks if t != "smmlv"]

    # stat
    stat = toks[0] if toks and toks[0] in STATS else None
    if not stat:
        return {
            "col": name, "stat": None, "tipo": None, "canal": None, "operacion": None,
            "window": window, "is_smmlv": is_smmlv
        }

    toks = toks[1:]  # remove stat

    # tipo: depende si stat == trx o no
    if stat == "trx":
        tipo = toks[0] if toks else None  # mnt/cnt/...
        toks = toks[1:]
    else:
        # esperamos "trx" como siguiente token
        if toks and toks[0] == "trx":
            toks = toks[1:]
        tipo = toks[0] if toks else None
        toks = toks[1:]

    remaining = "_".join(toks)  # aquí queda canal + operacion (o total)

    # detectar canal por diccionario (bloque completo)
    canal = None
    for ch in KNOWN_CHANNELS:
        if re.search(rf"(^|_){re.escape(ch)}(_|$)", remaining):
            canal = ch
            break

    # definir operacion = resto tras quitar canal (si se detectó)
    operacion = None
    if canal:
        remaining2 = re.sub(rf"(^|_){re.escape(canal)}(_|$)", "_", remaining, count=1).strip("_")
        operacion = remaining2 if remaining2 else "none"
    else:
        # si no detecta canal, puede ser total u otros campos globales
        operacion = remaining if remaining else "none"

    return {
        "col": name,
        "stat": stat,            # trx / avg / sum / min / max / stddev
        "tipo": tipo,            # mnt / cnt / ...
        "canal": canal,          # canal detectado
        "canal_desc": CHANNEL_DICT.get(canal, None),
        "operacion": operacion,  # pag/trf/ret/cmpr/... o "total"
        "window": window,        # ult3/ult6/ult9/ult12/none
        "is_smmlv": is_smmlv,
    }

catalog = pd.DataFrame([parse_col(c) for c in feature_cols])
catalog.head(20)

,col,stat,tipo,canal,canal_desc,operacion,window,is_smmlv
0,trx_mnt_total,trx,mnt,None,None,total,none,False
1,trx_mnt_total_smmlv,trx,mnt,None,None,total,none,True
2,trx_cnt_total,trx,cnt,None,None,total,none,False
3,trx_cnt_app_per_pag,trx,cnt,app_per_pag,app_per_pag,none,none,False
4,trx_cnt_app_per_pag_ob,trx,cnt,app_per_pag,app_per_pag,ob,none,False
5,trx_cnt_app_per_tfy,trx,cnt,app_per_tfy,app_per_tfy,none,none,False
6,trx_cnt_app_per_trf,trx,cnt,app_per,app personas,trf,none,False
7,trx_cnt_app_per_ava,trx,cnt,app_per,app personas,ava,none,False
8,trx_cnt_app_per_ut_c_cr,trx,cnt,app_per_ut_c,app_per_ut_c,cr,none,False
9,trx_cnt_app_pyme_trf,trx,cnt,app_pyme,app pymes,trf,none,False


In [15]:
print("Con canal detectado:", catalog["canal"].notna().sum())
print("Sin canal detectado:", catalog["canal"].isna().sum())

print("\nTop stats:")
display(catalog["stat"].value_counts())

print("\nTop tipo:")
display(catalog["tipo"].value_counts())

print("\nTop windows:")
display(catalog["window"].value_counts())

print("\nsmmlv:")
display(catalog["is_smmlv"].value_counts())

Con canal detectado: 4080
Sin canal detectado: 175

Top stats:


avg       816
max       816
stddev    816
sum       816
min       789
trx       202
Name: stat, dtype: int64


Top tipo:


mnt     2836
cnt     1418
tipo       1
Name: tipo, dtype: int64


Top windows:


ult3     1017
ult6     1017
ult9     1011
ult12    1008
none      202
Name: window, dtype: int64


smmlv:


False    2837
True     1418
Name: is_smmlv, dtype: int64

In [16]:
# Top canales por cantidad de columnas
top_canales = (catalog.dropna(subset=["canal"])
               .groupby(["canal","canal_desc"])["col"]
               .count()
               .sort_values(ascending=False)
               .head(30)
               .reset_index(name="n_cols"))
top_canales

,canal,canal_desc,n_cols
0,gr,gr,693
1,svp,sucursal virtual personas,372
2,cajero,cajero,315
3,suc_fis,sucursal fisica,252
4,pos,pos,252
5,app_per,app personas,189
6,cr_bcrio,cr_bcrio,189
7,rec_e_v,rec_e_v,189
8,app_per_pag,app_per_pag,126
9,sve_pag,sve_pag,126


In [17]:
# Top operaciones (las más frecuentes)
top_ops = (catalog["operacion"].value_counts().head(30).to_frame("n_cols"))
top_ops

,n_cols
can,882
trf,630
cmpr,378
pag,378
none,378
ret,315
ava,246
ob,189
cr,180
pag_ob,126


In [18]:
# Matriz de combinaciones por (stat, tipo, window)
combo = (catalog.groupby(["stat","tipo","window"])["col"]
         .count()
         .sort_values(ascending=False)
         .reset_index(name="n_cols"))
combo.head(30)

,stat,tipo,window,n_cols
0,max,mnt,ult3,136
1,sum,mnt,ult3,136
2,stddev,mnt,ult3,136
3,stddev,mnt,ult6,136
4,stddev,mnt,ult9,136
5,max,mnt,ult9,136
6,max,mnt,ult6,136
7,max,mnt,ult12,136
8,sum,mnt,ult12,136
9,stddev,mnt,ult12,136


In [19]:
# Construir pares: (col_pesos, col_smmlv)
pairs = []
for c in feature_cols:
    if "smmlv" in c:
        base = c.replace("_smmlv", "")
        if base in df_canales_sample.columns:
            pairs.append((base, c))

print("Pares base vs smmlv encontrados:", len(pairs))
pairs[:10]

Pares base vs smmlv encontrados: 1418


[('trx_mnt_total', 'trx_mnt_total_smmlv'),
 ('trx_mnt_app_per_pag', 'trx_mnt_app_per_pag_smmlv'),
 ('trx_mnt_app_per_pag_ob', 'trx_mnt_app_per_pag_ob_smmlv'),
 ('trx_mnt_app_per_tfy', 'trx_mnt_app_per_tfy_smmlv'),
 ('trx_mnt_app_per_trf', 'trx_mnt_app_per_trf_smmlv'),
 ('trx_mnt_app_per_ava', 'trx_mnt_app_per_ava_smmlv'),
 ('trx_mnt_app_per_ut_c_cr', 'trx_mnt_app_per_ut_c_cr_smmlv'),
 ('trx_mnt_app_pyme_trf', 'trx_mnt_app_pyme_trf_smmlv'),
 ('trx_mnt_bill_mvl_cmpr', 'trx_mnt_bill_mvl_cmpr_smmlv'),
 ('trx_mnt_btn_bco_trf', 'trx_mnt_btn_bco_trf_smmlv')]

In [20]:
# Correlación (sobre la muestra).
# Nota: si hay demasiados ceros, puede dar NaN; filtramos casos con varianza.
corr_rows = []
for base, sm in pairs[:200]:  # limita para que no sea pesado; luego puedes aumentar
    x = pd.to_numeric(df_canales_sample[base], errors="coerce")
    y = pd.to_numeric(df_canales_sample[sm], errors="coerce")
    
    if x.var(skipna=True) == 0 or y.var(skipna=True) == 0:
        continue
    
    corr = x.corr(y)
    corr_rows.append({"base": base, "smmlv": sm, "corr": corr})

corr_df = pd.DataFrame(corr_rows).sort_values("corr", ascending=False)
corr_df.head(20)

,base,smmlv,corr
108,avg_trx_mnt_gr_rcg_ult6,avg_trx_mnt_gr_rcg_smmlv_ult6,1.0
54,avg_trx_mnt_sve_pag_ob_ult3,avg_trx_mnt_sve_pag_ob_smmlv_ult3,1.0
27,trx_mnt_can_sve,trx_mnt_can_sve_smmlv,1.0
94,avg_trx_mnt_svp_dvs_ult6,avg_trx_mnt_svp_dvs_smmlv_ult6,1.0
6,trx_mnt_cajero_ava,trx_mnt_cajero_ava_smmlv,1.0
68,avg_trx_mnt_gr_rcg_ult3,avg_trx_mnt_gr_rcg_smmlv_ult3,1.0
16,trx_mnt_sve_pag_ob,trx_mnt_sve_pag_ob_smmlv,1.0
65,avg_trx_mnt_can_sve_ult3,avg_trx_mnt_can_sve_smmlv_ult3,1.0
56,avg_trx_mnt_svp_trf_ult3,avg_trx_mnt_svp_trf_smmlv_ult3,1.0
28,trx_mnt_can_svp,trx_mnt_can_svp_smmlv,1.0


In [21]:
out_dir = (Path.cwd() / "outputs")
out_dir.mkdir(exist_ok=True)

out_path = out_dir / "canales_columns_catalog.csv"
catalog.to_csv(out_path, index=False, encoding="utf-8")

print("✅ Catálogo guardado en:", out_path)

✅ Catálogo guardado en: c:\Users\serrios\01_prueba_analitico_4\op_cobro\notebooks\outputs\canales_columns_catalog.csv


<h3> Conclusiones: </h3>

- La tabla `raw.canales` es altamente dimensional y presenta una explosión combinatoria por la estructura de nombres:
  `stat` × `tipo (mnt/cnt)` × `canal` × `operacion` × `window` × `smmlv`. Esto genera miles de variables redundantes. 

- Se identificó duplicidad significativa por la presencia de versiones `_smmlv` (montos normalizados en salarios mínimos) frente a montos en pesos. Para el MVP de modelado, se considera `_smmlv` como redundante y candidato a descartarse para reducir multicolinealidad y complejidad. 

- Se evidenciaron inconsistencias de parsing en el catálogo generado: algunos “canales” quedan absorbidos por la operación
  (por ejemplo, `app_per_pag`, `app_per_tfy`, `app_per_ut_c`) y también aparecen casos tipo `can_suc_tel`. Esto confirma la necesidad de normalizar reglas de extracción canal/operación en el pipeline productivo. [1](https://bancolombia-my.sharepoint.com/personal/serrios_bancolombia_com_co1/_layouts/15/Doc.aspx?sourcedoc=%7B8DF8D393-093A-4D16-8ED5-A761089A30E3%7D&file=output_analisis_canales.xlsx&action=default&mobileredirect=true)

- Existen operaciones compuestas relevantes (`pag_ob`, `ut_c_cr`) que deben tratarse como sufijos de operación (no como parte del canal) para agrupar correctamente señales por canal. 

- A partir de lo anterior, la estrategia de producción será:
  1) Normalizar canal/operación (incluyendo operaciones compuestas y `can_total`),
  2) Reducir dimensionalidad seleccionando un set pequeño de ventanas y estadísticas,
  3) Generar señales analíticas compactas (120–150 features) con Spark en `pipelines/processing/features/canales_features.py`.

In [22]:
from pathlib import Path
from mlflow import spark
from pyspark.sql import functions as F

# Tabla raw
df = spark.table("raw.canales")

KEYS = ["num_doc", "obl17", "f_analisis"]

# Reglas MVP
KEEP_WINDOWS = {"ult3", "ult12"}
KEEP_STATS_PREFIX = ("sum_trx_", "avg_trx_")  # stats MVP
DROP_SMMLV = True

# Meta
TARGET_N = 140  # dentro del rango 120-150

AttributeError: module 'mlflow.spark' has no attribute 'table'